In [1]:
import csv
import pickle
import numpy

In [2]:
labels = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 
          'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']

In [3]:
mids = ['/m/0cmf2', '/m/0199g', '/m/015p6', '/m/019jd', '/m/04dr76w', '/m/01bjv', '/m/0k4j', 
        '/m/01yrx', '/m/01mzpv', '/m/01xq0k1', '/m/0h8n5zk', '/m/0bt9lr', '/m/03k3r', '/m/04_sv', 
        '/m/01g317', '/m/03fp41', '/m/07bgp', '/m/03m3pdh', '/m/07jdr', '/m/02522']

In [4]:
len(labels), len(mids), 

(20, 20)

In [5]:
label2mid = {}
for i in range(20):
    label2mid[labels[i]] = mids[i]

In [6]:
det_results = {}
pick = False
with open('/home/alireza/aida/rootdir/Ram/AIDA_detections_on_LDC2018E62_USC_IRIS/AIDA_detections_bbox_coordinates_on_keyframes_LDC2018E62_USC_IRIS.tab', 'r') as fin:
    for line in fin:
        if not pick:
            pick = True
            continue
            
        docid, entev, label, x1, y1, x2, y2, score = line.split()
        
        label = label.split('/')[-1]
        if label not in label2mid:
            continue
        
        val = {
            'label': label2mid[label],
            'score': float(score),
            'bbox': numpy.asarray([x1, y1, x2, y2], dtype='int32'),
            'model': 'pascal',            
        }
        
        if det_results.get(docid) is None:
            det_results[docid] = []
            
        det_results[docid].append(val)

In [10]:
kfid_to_docid = {}

with open('../../../../data/eval_m9/msb.txt', 'r') as fin:
    for line in fin:
        row = line.split()

        docid = row[1]
        kfid = row[0] + '_' + docid.split('_')[-1]
        
        kfid_to_docid[kfid] = docid


In [17]:
det_results_2 = {}
for key in det_results:
    if key in kfid_to_docid:
        det_results_2[kfid_to_docid[key]] = det_results[key]


In [13]:
len(det_results), len(det_results_2)

(62858, 62292)

In [14]:
with open('../../temp/m9_keyframe_imgsize_1.pkl', 'rb') as fin:
    image_shape = pickle.load(fin)

In [19]:
det_results_3 = {}
for imgid in det_results_2:
    det_results_3[imgid] = []
    for det in det_results_2[imgid]:
        det_results_3[imgid].append({
            'label': det['label'],
            'score': det['score'],
            'bbox': det['bbox'],
            'bbox_normalized': numpy.asarray([
                det['bbox'][0] / image_shape[imgid][0],
                det['bbox'][1] / image_shape[imgid][1],
                det['bbox'][2] / image_shape[imgid][0],
                det['bbox'][3] / image_shape[imgid][1],
            ]),
            'model': 'pascal',                       
        })

In [20]:
with open('../../results/det_results_ram_m9_keyframe_1', 'wb') as fout:
    pickle.dump(det_results_3, fout)